In [1]:
import pandas as pd
import keras
import numpy as np
from scipy.sparse import hstack


Using TensorFlow backend.


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] ="1"

In [29]:
#!python -m pytest --benchmark-only

In [4]:
import tensorflow as tf
from keras import layers

In [5]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfVectorizer
vectorizer = CountVectorizer(ngram_range=(2, 2), max_features=30,stop_words='english',lowercase=True)
#vectorizer = CountVectorizer(max_features=200,stop_words='english',lowercase=True)

In [4]:
data = pd.read_csv('corpus.csv')
from sklearn.preprocessing import MinMaxScaler
pos_scaler = MinMaxScaler()
neg_scaler = MinMaxScaler()
data['pos_scaled'] = pos_scaler.fit_transform(data[['pos']] )
data['neg_scaled'] = pos_scaler.fit_transform(data[['neg']] )

In [8]:
stopwords = [i for i in stopwords.words('english')]

In [9]:
import re
data['text']= [i.lower()  for i in data['text'] if i not in stopwords]
data['text'] = data['text'].apply((lambda c: re.sub('[^a-zA-z0-9\s]','',c)))

In [10]:
vectorizer.fit(data['text'])
X = vectorizer.fit_transform(data['text'])
X = hstack((X,np.array(data['pos_scaled'])[:,None]))
X = hstack((X,np.array(data['neg_scaled'])[:,None]))
X = X.todense()

In [11]:
#Tokenizers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
texts=data['text']
tokenizer = Tokenizer(num_words=None) 
tokenizer.fit_on_texts(texts)
#print( tokenizer.texts_to_sequences(texts)) 
#print( tokenizer.texts_to_matrix(texts))  

In [12]:
max_document_length=200
tokenizer = Tokenizer(num_words=200,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts.values)
 #截断补齐
x=pad_sequences(sequences, maxlen=max_document_length)

In [13]:
from keras.layers import Flatten
def baseline_model():
    model = Sequential()
    model.add(Dense(32, input_dim=32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(2, activation='softmax'))
    # Compile model
    model.compile(loss= 'categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])
    return model

In [14]:
stars=data['stars_y'].apply(lambda x:0 if x<3 else 1)

In [15]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,LSTM,Embedding
from keras.optimizers import SGD
from sklearn.model_selection import StratifiedKFold, cross_val_score
from keras.wrappers.scikit_learn import KerasClassifier
clf = KerasClassifier(build_fn=baseline_model, epochs=10, batch_size=12, verbose=1)
#使用5折交叉验证
scores = cross_val_score(clf, X, stars, cv=2, scoring='f1_micro')
# print scores
print("f1_micro: %0.3f (+/- %0.3f)" % (scores.mean(), scores.std() * 2))

Epoch 1/10
25677/25677 [==============================] - 1s 50us/step - loss: 0.4934 - accuracy: 0.7544
Epoch 2/10
25677/25677 [==============================] - 1s 48us/step - loss: 0.3914 - accuracy: 0.8186
Epoch 3/10
25677/25677 [==============================] - 1s 47us/step - loss: 0.3416 - accuracy: 0.8519
Epoch 4/10
25677/25677 [==============================] - 1s 49us/step - loss: 0.3227 - accuracy: 0.8621
Epoch 5/10
25677/25677 [==============================] - 1s 48us/step - loss: 0.3141 - accuracy: 0.8654
Epoch 6/10
25677/25677 [==============================] - 1s 47us/step - loss: 0.3096 - accuracy: 0.8682
Epoch 7/10
25677/25677 [==============================] - 1s 48us/step - loss: 0.3061 - accuracy: 0.8675
Epoch 8/10
25677/25677 [==============================] - 1s 47us/step - loss: 0.3052 - accuracy: 0.8699
Epoch 9/10
25677/25677 [==============================] - 1s 47us/step - loss: 0.3026 - accuracy: 0.8705
Epoch 10/10
25677/25677 [==============================

In [16]:
from keras.layers import Softmax
def lstm_mdl():
    model = Sequential()
    model.add(Embedding(128, 128,input_length = X.shape[1]))
    model.add(LSTM(128, recurrent_dropout=0.2,dropout = 0.2))
    model.add(Dense(2,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    print(model.summary())
    return model


In [17]:
X.shape

(51354, 32)

In [18]:
stars.shape

(51354,)

In [19]:
clf_x = KerasClassifier(build_fn=lstm_mdl, epochs=10, batch_size=64, verbose=1)
#使用5折交叉验证
scores = cross_val_score(clf_x, X, stars, cv=5, scoring='f1_micro')
print("f1_micro: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 32, 128)           16384     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 258       
Total params: 148,226
Trainable params: 148,226
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
41083/41083 [==============================] - 16s 390us/step - loss: 0.5583 - accuracy: 0.7476
Epoch 2/10
41083/41083 [==============================] - 16s 379us/step - loss: 0.5470 - accuracy: 0.7476
Epoch 3/10
41083/41083 [==============================] - 16s 378us/step - loss: 0.5441 - accuracy: 0.7478
Epoch 4/10
41083/41083 [============================

In [20]:
from sklearn.model_selection import train_test_split
Xtr, Xva, Ytr, Yva = train_test_split(X, stars, test_size=0.33, random_state=42)

In [21]:
from xgboost import XGBRegressor,XGBClassifier
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,confusion_matrix,plot_confusion_matrix
base = XGBClassifier(use_label_encoder= False,base_score = 0.30)
base.fit(Xtr,Ytr)

XGBClassifier(base_score=0.3, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, use_label_encoder=False, verbosity=1)

In [22]:
print(recall_score(base.predict(Xva),Yva))
print(accuracy_score(base.predict(Xva),Yva))
print(precision_score(base.predict(Xva),Yva))
print(f1_score(base.predict(Xva),Yva))

0.9102236668988468
0.8725438130642592
0.9216362354047488
0.9158944007476053


In [23]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(class_weight={0:1.4,1:1})

In [24]:
lr.fit(Xtr,Ytr)

LogisticRegression(C=1.0, class_weight={0: 1.4, 1: 1}, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [25]:
print(recall_score(lr.predict(Xva),Yva))
print(accuracy_score(lr.predict(Xva),Yva))
print(precision_score(lr.predict(Xva),Yva))
print(f1_score(lr.predict(Xva),Yva))

0.9186266771902131
0.8729568655219213
0.9120758561241282
0.9153395462231136


In [27]:
from keras.layers import Bidirectional
def mdl_rnn_bidir():
    model = Sequential()
    model.add(Embedding(128, 128,input_length = X.shape[1]))
    model.add(Bidirectional(LSTM(128, recurrent_dropout=0.2,dropout = 0.2)))
    model.add(Dense(2,activation='sigmoid'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    print(model.summary())
    return model

In [28]:
clf_x = KerasClassifier(build_fn=mdl_rnn_bidir, epochs=5, batch_size=64, verbose=1)
#使用n折交叉验证
scores = cross_val_score(clf_x, X, stars, cv=3, scoring='f1_micro')
# print scores
print("f1_micro: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 32, 128)           16384     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               263168    
_________________________________________________________________
dense_10 (Dense)             (None, 2)                 514       
Total params: 280,066
Trainable params: 280,066
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
34236/34236 [==============================] - 21s 607us/step - loss: 0.5564 - accuracy: 0.7486
Epoch 2/5
34236/34236 [==============================] - 20s 591us/step - loss: 0.5403 - accuracy: 0.7504
Epoch 3/5
34236/34236 [==============================] - 20s 591us/step - loss: 0.5372 - accuracy: 0.7538
Epoch 4/5
34236/34236 [==============================] 